In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = 'meta-llama/Llama-3.2-3B'

tokenizer = AutoTokenizer.from_pretrained('/home/compiling-ganesh/24m0797/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062', local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [97]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [146]:
prompt = """1<op>1=2, 1<op>5=6, 3<op>2=5, 3<op>5=8, 4<op>4=8, 4<op>4="""
prompt2 = """4<op>4="""

model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
model_inputs2 = tokenizer([prompt2], return_tensors="pt").to(model.device)

input_length = model_inputs.input_ids.shape[1]
input_length2 = model_inputs2.input_ids.shape[1]

In [17]:
device = 'cuda'
model = model.to(device)

In [144]:
generated_ids = model.generate(**model_inputs, max_new_tokens=1, do_sample=False)
print(tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forward hook on LlamaAttention
Output: torch.Size([1, 52, 3072])
9


In [154]:
generated_ids2 = model.generate(**model_inputs2, max_new_tokens=1, do_sample=False)
print(tokenizer.batch_decode(generated_ids2[:, input_length2:], skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forward hook on LlamaAttention
Named Parameters:
  q_proj.weight: torch.Size([3072, 3072])
  k_proj.weight: torch.Size([1024, 3072])
  v_proj.weight: torch.Size([1024, 3072])
  o_proj.weight: torch.Size([3072, 3072])
Output shape: torch.Size([1, 7, 3072])
4


In [153]:

def my_forward_hook(module, ip, op):
        print(f"Forward hook on {module.__class__.__name__}")
        print("Named Parameters:")
        for name, param in module.named_parameters():
            print(f"  {name}: {param.shape}")
        print(f"Output shape: {op[0].shape}")
        # You can also modify the output here, e.g., return output * 2

def my_forward_pre_hook(module, ip):
        print(f"Forward hook on {module.__class__.__name__}")
        print(f"Input: {ip}")
        # You can also modify the output here, e.g., return output * 2
    
try:
    handle.remove()
finally:
    handle = model.model.layers[0].self_attn.register_forward_hook(my_forward_hook)

LlamaAttention(
  (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
  (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
  (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
  (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
)